In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 04f9090a-1e95-4c2f-8404-074de3278152, 12, Finished, Available, Finished)

In [ ]:
%%sql
create table if not exists Gold_Customer
(
  Customer_ID	string	,
Customer_Name	string	,
Segment	string	,
City	string	,
State	string	,
Country	string	,
Region	string	,
Created_TS	timestamp	,
Modified_TS	timestamp	  

)
using delta

StatementMeta(, 04f9090a-1e95-4c2f-8404-074de3278152, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [ ]:
Max_Date=spark.sql("""select coalesce(max(Modified_TS),"1900-01-01") from gold_Customer""").first()[0]

StatementMeta(, 04f9090a-1e95-4c2f-8404-074de3278152, 14, Finished, Available, Finished)

In [ ]:
Max_Date

StatementMeta(, 04f9090a-1e95-4c2f-8404-074de3278152, 15, Finished, Available, Finished)

'1900-01-01'

In [ ]:
df_bronze=spark.read.table("Bronze_Sales") # instead of spaek sql way--> spark.sql, we use spark.read pyspark way

StatementMeta(, 04f9090a-1e95-4c2f-8404-074de3278152, 16, Finished, Available, Finished)

In [ ]:
df_final=df_bronze.selectExpr("Customer_ID","Customer_Name","Segment","City","State",\
                   "Country","Region")\
                   .where(col("Modified_TS")>Max_Date)\
                   .drop_duplicates()

StatementMeta(, 04f9090a-1e95-4c2f-8404-074de3278152, 17, Finished, Available, Finished)

In [ ]:
df_final.createOrReplaceTempView("ViewCustomer")

StatementMeta(, 04f9090a-1e95-4c2f-8404-074de3278152, 18, Finished, Available, Finished)

In [ ]:
%%sql
Merge into Gold_Customer as gc
using ViewCustomer as vc
on gc.Customer_ID=vc.Customer_ID
when matched then 
update set
gc.Customer_Name	=	vc.Customer_Name	,
gc.Segment	=	vc.Segment	,
gc.City	=	vc.City	,
gc.State	=	vc.State	,
gc.Country	=	vc.Country	,
gc.Region	=	vc.Region	,
gc.Modified_TS	=	current_timestamp()	

when not matched then insert
(
gc.Customer_ID	,
gc.Customer_Name	,
gc.Segment	,
gc.City	,
gc.State	,
gc.Country	,
gc.Region	,
gc.Created_TS	,
gc.Modified_TS	
)
values
(
vc.Customer_ID	,
vc.Customer_Name	,
vc.Segment	,
vc.City	,
vc.State	,
vc.Country	,
vc.Region	,
current_timestamp(),
current_timestamp
)

StatementMeta(, 04f9090a-1e95-4c2f-8404-074de3278152, 20, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>